In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 定义简单的两层神经网络
class RZDecoderNet(nn.Module):
    def __init__(self, input_size):
        super(RZDecoderNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# 从文件中加载数据
def load_data_from_file(file_path, bit_length):
    inputs = []
    outputs = []
    with open(file_path, 'r') as f:
        for line in f:
            values = [float(x) for x in line.strip().split()]
            inputs.append(values[:bit_length])  # 前 bit_length 个值为输入信号
            outputs.append(values[bit_length])  # 最后一个值为输出标签

    inputs = torch.tensor(inputs, dtype=torch.float32)
    outputs = torch.tensor(outputs, dtype=torch.float32).unsqueeze(1)  # 添加维度以匹配网络输出
    return inputs, outputs

# 训练 RZ 解码网络
def train_rz_decoder(model, train_loader, num_epochs=20, learning_rate=0.01):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# 解码 RZ 信号
def rz_decode_with_nn(model, signal, bit_length):
    model.eval()
    decoded_bits = []
    with torch.no_grad():
        for i in range(len(signal)):
            bit_window = signal[i].unsqueeze(0)
            output = model(bit_window)
            decoded_bits.append(1 if output >= 0.5 else 0)
    return torch.tensor(decoded_bits)

# 参数
bit_length = 1  # 每个比特的长度

# 从文件加载训练和测试数据
train_file_path = r'E:\gitprogram\data\testdata\train.txt'
test_file_path = r'E:\gitprogram\data\testdata\test.txt'

# 加载数据
train_inputs, train_labels = load_data_from_file(train_file_path, bit_length)
test_inputs, test_labels = load_data_from_file(test_file_path, bit_length)

# 创建数据加载器
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

# 初始化网络
input_size = bit_length
model = RZDecoderNet(input_size)

# 训练网络
train_rz_decoder(model, train_loader, num_epochs=200, learning_rate=0.01)

# 使用测试数据进行解码
decoded_bits = rz_decode_with_nn(model, test_inputs, bit_length)
print("Decoded bits:", decoded_bits)
print("True bits:   ", test_labels.squeeze().int())


Epoch [1/200], Loss: 0.7956
Epoch [2/200], Loss: 0.7744
Epoch [3/200], Loss: 0.7541
Epoch [4/200], Loss: 0.7347
Epoch [5/200], Loss: 0.7162
Epoch [6/200], Loss: 0.6988
Epoch [7/200], Loss: 0.6823
Epoch [8/200], Loss: 0.6668
Epoch [9/200], Loss: 0.6522
Epoch [10/200], Loss: 0.6390
Epoch [11/200], Loss: 0.6263
Epoch [12/200], Loss: 0.6141
Epoch [13/200], Loss: 0.6021
Epoch [14/200], Loss: 0.5904
Epoch [15/200], Loss: 0.5789
Epoch [16/200], Loss: 0.5674
Epoch [17/200], Loss: 0.5560
Epoch [18/200], Loss: 0.5447
Epoch [19/200], Loss: 0.5333
Epoch [20/200], Loss: 0.5222
Epoch [21/200], Loss: 0.5112
Epoch [22/200], Loss: 0.5000
Epoch [23/200], Loss: 0.4889
Epoch [24/200], Loss: 0.4777
Epoch [25/200], Loss: 0.4665
Epoch [26/200], Loss: 0.4553
Epoch [27/200], Loss: 0.4440
Epoch [28/200], Loss: 0.4328
Epoch [29/200], Loss: 0.4215
Epoch [30/200], Loss: 0.4103
Epoch [31/200], Loss: 0.3990
Epoch [32/200], Loss: 0.3878
Epoch [33/200], Loss: 0.3770
Epoch [34/200], Loss: 0.3668
Epoch [35/200], Loss: 0